In [1]:
import numpy as np
import pandas as pd
from sklearn import linear_model
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sqlalchemy import create_engine

import warnings
warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
housing_df = pd.read_sql_query('select * from houseprices',con=engine)


engine.dispose()

In [2]:
#Create dummies for the categorical features

housing_df = pd.concat([housing_df,pd.get_dummies(housing_df.kitchenqual, prefix='kitchenqual', drop_first=True)], axis=1)
housing_df = pd.concat([housing_df,pd.get_dummies(housing_df.exterqual, prefix='exterqual', drop_first=True)], axis=1)

dummy_column_names = list(pd.get_dummies(housing_df.kitchenqual, prefix='kitchenqual', drop_first=True).columns)
dummy_column_names = dummy_column_names + list(pd.get_dummies(housing_df.exterqual, prefix='exterqual', drop_first=True).columns)

In [3]:
#Setting variables for model
X = housing_df[['overallqual', 'grlivarea', 'garagecars'] + dummy_column_names]
Y = housing_df.saleprice

In [4]:
#Running regression model

X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.779
Model:                            OLS   Adj. R-squared:                  0.777
Method:                 Least Squares   F-statistic:                     567.3
Date:                Sat, 17 Aug 2019   Prob (F-statistic):               0.00
Time:                        20:20:14   Log-Likelihood:                -17443.
No. Observations:                1460   AIC:                         3.491e+04
Df Residuals:                    1450   BIC:                         3.496e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const           5.846e+04   1.13e+04      5.189      0.000    3.64e+04    8.06e+04
overallqual     1.768e+04   1236.605     14.299      0.000    1.53e+04    2.01e+04
grlivarea         48.6266      2.369     20.530      0.000      43.980      53.273
garagecars      1.856e+04   1714.192     10.830      0.000    1.52e+04    2.19e+04
kitchenqual_Fa -5.282e+04   8556.765     -6.172      0.000   -6.96e+04    -3.6e+04
kitchenqual_Gd -4.101e+04   4909.002     -8.354      0.000   -5.06e+04   -3.14e+04
kitchenqual_TA -5.139e+04   5510.696     -9.326      0.000   -6.22e+04   -4.06e+04
exterqual_Fa   -5.861e+04   1.32e+04     -4.438      0.000   -8.45e+04   -3.27e+04
exterqual_Gd   -4.153e+04   6564.259     -6.326      0.000   -5.44e+04   -2.87e+04
exterqual_TA    -5.43e+04   7245.726     -7.494      0.000   -6.85e+04   -4.01e+04
==============================================================================
Omnibus:                      362.267   Durbin-Watson:                   2.002
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            21126.953
Skew:                           0.055   Prob(JB):                         0.00
Kurtosis:                      21.635   Cond. No.                     2.69e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.69e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

#### Interpret the results.
The constant/bias term is 58460. <br> 
For every point increase in overallqual, the sales price increases 17680.  <br> 
For every point increase in grlivarea, the sales price increases by 48.63.<br> 
For every point increase in garagecars, the sales price increases by 18560.  <br> 
For every point increase in kitchenqual_Fa, the sales price decreases by 52820.
For every point increase in kitchenqual_Gd, the sales price decreases by 41010.
For every point increase in kitchenqual_TA, the sales price decreases by 51390. <br> 
So kitchenqual_Fa results in the lowest housing sales price in the kitchenqual categorical variable, while kitchenqual_Gd is the highest. All categories are relative to the dropped kitchenqual_Ex variable.  <br>  
For every point increase in exterqual_Fa, the sales price decreases by 58610. <br>
For every point increase in exterqual_Gd, the sales price decreases by 41530. <br>
For every point increase in exterqual_ the sales price decreases by 54300.  <br>
The sales price is the highest in exterqual_Gd and lowest in exterqual_Fa compared to the dropped dummy variable kitchenqual_Ex.  All categories are relative to the dropped exterqual_Ex variable.    

#### Which features are statistically significant, and which are not?
All of the features are statistically significant in the model. 

#### Which features have a more prominent effect on house prices?
Overallqual and garagecars have the most prominent effect on avg housing sales price.  Other than overallqual, garagecars, and grlivarea, all of the other features are negative coefficients.  

#### Do the results sound reasonable to you? If not, try to explain the potential reasons.
The results seem very reasonable to me.  All of the features that increase the value make sense.  All of the features that decrease the value make sense when compared to the dropped dummy variable.  
